In [29]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.morphology import disk, binary_closing, remove_small_objects
import matplotlib.pyplot as plt

# Define paths
dataset_path = "./dataset/train"
test_path = "./dataset/test"

def load_data(dataset_path):
    data = []
    labels = []

    for category in os.listdir(dataset_path):  # car, bus, truck
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                # Read and preprocess the image
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))  # Resize for uniformity
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                edges = cv2.Canny(gray, gray.mean() - gray.std(), gray.mean() + gray.std())  # Apply Canny edge detection

                binarized_edges = edges > 0  # Binarize the edge-detected image
                binarized_edges = remove_small_objects(binarized_edges, min_size=3)

                closed_edge = binary_closing(binarized_edges, disk(17))
                filled_segment = cv2.morphologyEx(closed_edge.astype(np.uint8), cv2.MORPH_CLOSE, None)

                # Apply the mask to the image
                if len(img.shape) == 3:
                    obj_img = img * filled_segment[:, :, np.newaxis]
                else:
                    obj_img = img * filled_segment

                # Histogram of Oriented Gradients (HOG)
                winSize = (64, 64)
                blockSize = (16, 16)
                blockStride = (8, 8)
                cellSize = (8, 8)
                nbins = 9
                hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
                hog_features = hog.compute(obj_img).flatten()
                        
                # Flatten the edge-detected image as features
                data.append(hog_features)
                labels.append(category)

    return np.array(data), np.array(labels)

In [30]:
# Load the dataset
X_train, y = load_data(dataset_path)
X_test, y_test = load_data(test_path)

# Encode labels (car -> 0, bus -> 1, truck -> 2)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)

# Split dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train an SVM classifier
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = svm_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Classification Report:
              precision    recall  f1-score   support

         Bus       1.00      0.67      0.80         3
         Car       1.00      0.67      0.80         3
  Motorcycle       1.00      1.00      1.00         3
       Truck       0.87      1.00      0.93        13

    accuracy                           0.91        22
   macro avg       0.97      0.83      0.88        22
weighted avg       0.92      0.91      0.90        22



In [38]:
from joblib import Parallel, delayed 
import joblib 

# Save the model
joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(label_encoder, 'svm_label_encoder.pkl')

#Load the model
loaded_model = joblib.load('svm_model.pkl')


In [39]:
# Prediction function
def predict_image(image_path):
    img = cv2.imread(image_path)
    print(image_path)
    img = cv2.resize(img, (128, 128))  # Resize for uniformity
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, gray.mean() - gray.std(), gray.mean() + gray.std())  # Apply Canny edge detection

    binarized_edges = edges > 0  # Binarize the edge-detected image
    binarized_edges = remove_small_objects(binarized_edges, min_size=3)

    closed_edge = binary_closing(binarized_edges, disk(17))
    filled_segment = cv2.morphologyEx(closed_edge.astype(np.uint8), cv2.MORPH_CLOSE, None)
    
    # Apply the mask to the image
    if len(img.shape) == 3:
        obj_img = img * filled_segment[:, :, np.newaxis]
    else:
        obj_img = img * filled_segment

    # Histogram of Oriented Gradients (HOG)
    winSize = (64, 64)
    blockSize = (16, 16)
    blockStride = (8, 8)
    cellSize = (8, 8)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    hog_features = [hog.compute(obj_img).flatten()]
                        
    # Predict the class and confidence
    predicted_class_index = loaded_model.predict(hog_features)[0]
    confidence = np.max(loaded_model.predict_proba(hog_features))
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label, confidence

In [41]:
# Test with a new image
test_image_path = "./dataset/test/Bus/blue-bus.jpg"  # Replace with your test image path
predicted_label, confidence = predict_image(test_image_path)
print(f"The predicted label for the test image is: {predicted_label}")
print(f"Confidence level: {confidence:.2f}")

./dataset/test/Bus/blue-bus.jpg
The predicted label for the test image is: Bus
Confidence level: 0.61
